### Attempting to run regressions from systematics values extracted geometrically or from Kitanidis et al on galaxy density


In [27]:
import pandas as pd
import numpy as np
import pickle
import healpy as hp
import statsmodels.api as sm


In [2]:
### Setting Healpy Information
NSIDE = 256
NPIX = hp.nside2npix(NSIDE)
print(
    "Approximate resolution at NSIDE {} is {:.2} deg".format(
        NSIDE, hp.nside2resol(NSIDE, arcmin=True) / 60
    )
)

def raDec2thetaPhi(ra, dec):
    return (0.5 * np.pi - np.deg2rad(dec)), (np.deg2rad(ra))


Approximate resolution at NSIDE 256 is 0.23 deg


### Importing Galaxy Density Information

In [3]:
df_objects = pd.read_csv('../bricks_data/galaxy_catalogue_sample.csv')


In [4]:
df_objects.head()

,BrickID,ObjectID,RA,DEC,South,Target_type
0,389675,10,133.284363,10.315927,True,3
1,389675,11,133.284832,10.314327,True,2
2,389675,15,133.285109,10.294865,True,3
3,389675,21,133.285731,10.319199,True,3
4,389675,35,133.286350,10.168023,True,2


In [5]:
# LRG

df_LRG = df_objects[df_objects["Target_type"] == 1]
ra_LRG = df_LRG["RA"].to_numpy(copy=True)
dec_LRG = df_LRG["DEC"].to_numpy(copy=True)
theta, phi = raDec2thetaPhi(ra_LRG, dec_LRG)

print("Number of LRGs in Sample:", len(ra_LRG))

LRG_pixel_indices = hp.ang2pix(NSIDE, theta,phi)


# Finding out unique indices and how often they appear --> shows the density of LRGs in this pixel
(unique, counts) = np.unique(LRG_pixel_indices, return_counts=True)

print(unique)

mean_LRG = counts.mean()
print("Mean LRGs per Pixel:", mean_LRG)
# Calculating Densities for every block
density_LRG = (counts / mean_LRG) - 1
id_density = np.stack((unique, density_LRG), axis=1)
print(id_density)
df_lrg = pd.DataFrame(id_density, columns=['pixel_id', 'lrg_density'])
df_lrg.pixel_id = df_lrg.pixel_id.astype(int)
df_lrg.head()

Number of LRGs in Sample: 41904
[166098 167122 167123 ... 756988 756989 757472]
Mean LRGs per Pixel: 9.93692198245198
[[ 1.66098000e+05  3.08252195e-01]
 [ 1.67122000e+05 -8.99365216e-01]
 [ 1.67123000e+05  4.08886980e-01]
 ...
 [ 7.56988000e+05 -8.99365216e-01]
 [ 7.56989000e+05  1.61650439e+00]
 [ 7.57472000e+05 -4.96826079e-01]]


,pixel_id,lrg_density
0,166098,0.308252
1,167122,-0.899365
2,167123,0.408887
3,171167,-0.496826
4,172190,-0.496826


In [6]:
# LRG

df_ELG = df_objects[df_objects["Target_type"] == 2]
ra_ELG = df_ELG["RA"].to_numpy(copy=True)
dec_ELG = df_ELG["DEC"].to_numpy(copy=True)
theta, phi = raDec2thetaPhi(ra_ELG, dec_ELG)

print("Number of ELGs in Sample:", len(ra_ELG))

ELG_pixel_indices = hp.ang2pix(NSIDE, theta,phi)


# Finding out unique indices and how often they appear --> shows the density of LRGs in this pixel
(unique, counts) = np.unique(ELG_pixel_indices, return_counts=True)

print(unique)

mean_ELG = counts.mean()
print("Mean ELGs per Pixel:", mean_ELG)
# Calculating Densities for every block
density_ELG = (counts / mean_ELG) - 1
id_density = np.stack((unique, density_ELG), axis=1)
print(id_density)
df_elg = pd.DataFrame(id_density, columns=['pixel_id', 'elg_density'])
df_elg.pixel_id = df_elg.pixel_id.astype(int)

print(df_elg.head())

Number of ELGs in Sample: 157118
[166098 167122 167123 ... 756502 756989 757472]
Mean ELGs per Pixel: 32.556568586821385
[[ 1.66098000e+05  7.50518718e-02]
 [ 1.67122000e+05 -9.07852697e-01]
 [ 1.67123000e+05 -4.16400412e-01]
 ...
 [ 7.56502000e+05 -5.69979251e-01]
 [ 7.56989000e+05  4.34454359e+00]
 [ 7.57472000e+05  1.36203363e-02]]
   pixel_id  elg_density
0    166098     0.075052
1    167122    -0.907853
2    167123    -0.416400
3    171167     0.412925
4    172190    -0.293537


In [7]:
df_QSO = df_objects[df_objects["Target_type"] == 3]
ra_QSO = df_QSO["RA"].to_numpy(copy=True)
dec_QSO = df_QSO["DEC"].to_numpy(copy=True)
theta, phi = raDec2thetaPhi(ra_QSO, dec_QSO)

print("Number of QSOs in Sample:", len(ra_QSO))

QSO_pixel_indices = hp.ang2pix(NSIDE, theta,phi)


# Finding out unique indices and how often they appear --> shows the density of LRGs in this pixel
(unique, counts) = np.unique(QSO_pixel_indices, return_counts=True)

print(unique)

mean_QSO = counts.mean()
print("Mean QSOs per Pixel:", mean_QSO)
# Calculating Densities for every block
density_QSO = (counts / mean_QSO) - 1
id_density = np.stack((unique, density_QSO), axis=1)
print(id_density)
df_qso = pd.DataFrame(id_density, columns=['pixel_id', 'qso_density'])
df_qso.pixel_id = df_qso.pixel_id.astype(int)

print(df_qso.head())

Number of QSOs in Sample: 180394
[166098 167122 167123 ... 756502 756989 757472]
Mean QSOs per Pixel: 36.92058943921408
[[ 1.66098000e+05  5.63211637e-02]
 [ 1.67122000e+05 -8.10403894e-01]
 [ 1.67123000e+05  1.10491480e-01]
 ...
 [ 7.56502000e+05 -8.64574210e-01]
 [ 7.56989000e+05  2.60232602e+00]
 [ 7.57472000e+05 -1.60360101e-01]]
   pixel_id  qso_density
0    166098     0.056321
1    167122    -0.810404
2    167123     0.110491
3    171167    -0.024934
4    172190    -0.133275


In [8]:
df = pd.merge(df_lrg, df_elg, how='outer', on='pixel_id')
df = pd.merge(df, df_qso, how='outer', on='pixel_id')
print(df.isna().sum())
print(df_lrg.shape, df_elg.shape, df_qso.shape, df.shape)

pixel_id         0
lrg_density    860
elg_density    251
qso_density    191
dtype: int64
(4217, 2) (4826, 2) (4886, 2) (5077, 4)


In [9]:
# Dropping all pixels where I do not have densities for all 3 types of galaxies
df.dropna(axis=0, inplace=True)
print(df.isna().sum())

pixel_id       0
lrg_density    0
elg_density    0
qso_density    0
dtype: int64


### Importing the Systematics Values and Appending them to the df.


#### Stellar Density

In [10]:
df_Stellar = pd.read_csv('../bricks_data/galaxy_catalogue_stellar.csv')
ra_stellar = df_Stellar["RA"].to_numpy(copy=True)
dec_stellar = df_Stellar["DEC"].to_numpy(copy=True)
theta, phi = raDec2thetaPhi(ra_stellar, dec_stellar)

print("Number of stellars in Sample:", len(ra_stellar))

stellar_pixel_indices = hp.ang2pix(NSIDE, theta,phi)


# Finding out unique indices and how often they appear --> shows the density of LRGs in this pixel
(unique, counts) = np.unique(stellar_pixel_indices, return_counts=True)

print(unique)

mean_stellar = counts.mean()
print("Mean stellars per Pixel:", mean_stellar)
# Calculating Densities for every block
density_stellar = (counts / mean_stellar) - 1
id_density = np.stack((unique, density_stellar), axis=1)
print(id_density)
df_stellar = pd.DataFrame(id_density, columns=['pixel_id', 'stellar_density'])
df_stellar.pixel_id = df_stellar.pixel_id.astype(int)

print(df_stellar.head())

print(df_stellar.isna().sum())

Number of stellars in Sample: 38979
[166098 167123 171167 ... 756502 756989 757472]
Mean stellars per Pixel: 9.718025430067314
[[ 1.66098000e+05 -6.91295313e-01]
 [ 1.67123000e+05  1.31917186e-01]
 [ 1.71167000e+05 -4.85492188e-01]
 ...
 [ 7.56502000e+05 -7.94196875e-01]
 [ 7.56989000e+05  2.18994843e+00]
 [ 7.57472000e+05 -5.88393750e-01]]
   pixel_id  stellar_density
0    166098        -0.691295
1    167123         0.131917
2    171167        -0.485492
3    172190        -0.794197
4    172191        -0.279689
pixel_id           0
stellar_density    0
dtype: int64


In [11]:
df = df.merge(df_stellar, how='inner', on='pixel_id')
print(df.isna().sum())
df.shape

pixel_id           0
lrg_density        0
elg_density        0
qso_density        0
stellar_density    0
dtype: int64


(3621, 5)

#### Kitanidis Systematics Values

In [12]:
with open('../bricks_data/pixel2systematics_kitanidis.pickle', 'rb') as f:
    kitanidis_dict = pickle.load(f)
    f.close()

print(len(kitanidis_dict.keys()))

1567


In [13]:
pixel_keys = kitanidis_dict.keys()

sys_list = []
for key in pixel_keys:
    ind_sys_list = kitanidis_dict[key]
    ind_sys_list.append(key)
    sys_list.append(ind_sys_list)

df_sys_kit = pd.DataFrame(sys_list, columns=['airmass_galaxy',
'fwhm_galaxy',
'ebv_galaxy',
'ccdnphotom_galaxy',

'ccdskysb_galaxy_g',
'ccdskysb_galaxy_r',
'ccdskysb_galaxy_z',
'exptime_galaxy_g',
'exptime_galaxy_r',
'exptime_galaxy_z',
'meansky_galaxy_g',
'meansky_galaxy_r',
'meansky_galaxy_z',
'galdepth_galaxy_g',
'galdepth_galaxy_r',
'galdepth_galaxy_z', 'pixel_id'])



In [14]:
df_sys_kit.head()

,airmass_galaxy,fwhm_galaxy,ebv_galaxy,ccdnphotom_galaxy,ccdskysb_galaxy_g,ccdskysb_galaxy_r,ccdskysb_galaxy_z,exptime_galaxy_g,exptime_galaxy_r,exptime_galaxy_z,meansky_galaxy_g,meansky_galaxy_r,meansky_galaxy_z,galdepth_galaxy_g,galdepth_galaxy_r,galdepth_galaxy_z,pixel_id
0,1.204206,4.569465,0.021082,273.100211,23.627050,22.730956,20.206251,90.0,90.0,90.0,2.390975e+08,6.489436e+08,2.384546e+09,23.111424,23.139025,22.097923,741107
1,1.069105,4.824085,0.010359,148.191320,23.601375,22.732608,20.000021,90.0,90.0,90.0,1.803126e+08,2.993781e+08,2.225294e+10,23.353823,23.320494,21.550398,662895
2,1.108757,4.413348,0.011393,154.348513,23.602864,22.701580,20.073285,90.0,90.0,90.0,2.952632e+08,8.687714e+08,1.152634e+10,23.062541,22.892199,21.675296,662913
3,1.104669,4.394738,0.011230,153.649709,23.613133,22.710532,20.111641,90.0,90.0,90.0,2.446651e+08,9.609978e+08,8.914973e+09,23.186001,22.788259,21.786543,663905
4,1.107077,4.439277,0.014093,150.531232,23.583476,22.647124,20.197871,90.0,90.0,90.0,3.645015e+08,6.979010e+08,9.072512e+09,22.836915,23.042679,21.724951,652792


In [15]:
df_kitanidis = df.merge(df_sys_kit, how='inner', on='pixel_id')
print(df_kitanidis.isna().sum())
df_kitanidis.dropna(axis=0, inplace=True)

df_kitanidis.head()


pixel_id             0
lrg_density          0
elg_density          0
qso_density          0
stellar_density      0
airmass_galaxy       0
fwhm_galaxy          0
ebv_galaxy           0
ccdnphotom_galaxy    0
ccdskysb_galaxy_g    3
ccdskysb_galaxy_r    0
ccdskysb_galaxy_z    0
exptime_galaxy_g     0
exptime_galaxy_r     0
exptime_galaxy_z     0
meansky_galaxy_g     0
meansky_galaxy_r     0
meansky_galaxy_z     0
galdepth_galaxy_g    0
galdepth_galaxy_r    0
galdepth_galaxy_z    0
dtype: int64


,pixel_id,lrg_density,elg_density,qso_density,stellar_density,airmass_galaxy,fwhm_galaxy,ebv_galaxy,ccdnphotom_galaxy,ccdskysb_galaxy_g,...,ccdskysb_galaxy_z,exptime_galaxy_g,exptime_galaxy_r,exptime_galaxy_z,meansky_galaxy_g,meansky_galaxy_r,meansky_galaxy_z,galdepth_galaxy_g,galdepth_galaxy_r,galdepth_galaxy_z
0,172191,0.509522,1.180820,0.787620,-0.279689,1.615369,4.563708,0.027940,617.485215,23.461968,...,19.328887,143.100496,139.892237,122.275265,0.199914,0.920791,2.869334,23.626332,22.942529,22.215109
1,173250,1.817774,2.133008,1.410579,1.263834,1.551244,4.652137,0.022238,997.009628,23.454327,...,19.084433,158.096504,82.525712,165.945433,0.203127,0.499488,3.490955,23.604491,22.966942,22.331463
2,174071,0.106983,0.750799,0.191747,-0.794197,1.406610,4.458577,0.018477,367.511248,23.441590,...,19.497361,128.317445,87.744524,106.219834,0.213351,0.564238,1.995409,23.598964,22.922293,22.342016
3,174554,0.106983,0.658651,0.083406,0.955130,2.301381,6.055535,0.131243,459.871429,23.432264,...,19.849928,200.000000,124.000000,221.028571,0.126744,0.333307,3.754301,24.007800,23.363166,22.229356
4,175096,1.515870,3.146629,1.329324,-0.485492,1.440663,4.587659,0.018841,373.382383,23.452281,...,19.589021,128.873803,83.743415,107.265548,0.293171,0.540003,6.617020,23.605227,22.900591,22.310904


In [16]:
print(df_kitanidis.isna().sum())

pixel_id             0
lrg_density          0
elg_density          0
qso_density          0
stellar_density      0
airmass_galaxy       0
fwhm_galaxy          0
ebv_galaxy           0
ccdnphotom_galaxy    0
ccdskysb_galaxy_g    0
ccdskysb_galaxy_r    0
ccdskysb_galaxy_z    0
exptime_galaxy_g     0
exptime_galaxy_r     0
exptime_galaxy_z     0
meansky_galaxy_g     0
meansky_galaxy_r     0
meansky_galaxy_z     0
galdepth_galaxy_g    0
galdepth_galaxy_r    0
galdepth_galaxy_z    0
dtype: int64


#### Geometric Systematics Values

In [17]:
with open('../bricks_data/pixel2systematics_geometric.pickle', 'rb') as f:
    geometric_dict = pickle.load(f)
    f.close()

print(len(geometric_dict.keys()))


455672


In [18]:
# Clear the dict of all pixels where we do not have galaxy density values

pixels = df.pixel_id.tolist()
pixel_keys = list(geometric_dict.keys())
for pix in pixel_keys:
    if pix in pixels:
        continue
    geometric_dict.pop(pix)
print(len(geometric_dict.keys()))


3621


In [19]:
sys_list = []
for key in geometric_dict.keys():
    ind_sys_list = geometric_dict[key]
    ind_sys_list.append(key)
    sys_list.append(ind_sys_list)

df_sys_geo = pd.DataFrame(sys_list, columns=['airmass_galaxy',
'fwhm_galaxy',
'ebv_galaxy',
'ccdnphotom_galaxy',

'ccdskysb_galaxy_g',
'ccdskysb_galaxy_r',
'ccdskysb_galaxy_z',
'exptime_galaxy_g',
'exptime_galaxy_r',
'exptime_galaxy_z',
'meansky_galaxy_g',
'meansky_galaxy_r',
'meansky_galaxy_z',
'galdepth_galaxy_g',
'galdepth_galaxy_r',
'galdepth_galaxy_z', 'pixel_id'])



In [20]:
df_sys_geo.head()

,airmass_galaxy,fwhm_galaxy,ebv_galaxy,ccdnphotom_galaxy,ccdskysb_galaxy_g,ccdskysb_galaxy_r,ccdskysb_galaxy_z,exptime_galaxy_g,exptime_galaxy_r,exptime_galaxy_z,meansky_galaxy_g,meansky_galaxy_r,meansky_galaxy_z,galdepth_galaxy_g,galdepth_galaxy_r,galdepth_galaxy_z,pixel_id
0,1.194180,4.416241,0.015881,193.029146,0.370143,0.354336,0.317393,1.40625,1.40625,1.40625,1.874880e+06,1.128373e+07,3.228007e-02,0.367989,0.359158,0.351488,729635
1,1.204914,4.487709,0.021031,263.930656,0.369211,0.355531,0.312636,1.40625,1.40625,1.40625,2.882011e+06,8.985916e+06,1.687051e+08,0.362452,0.362690,0.338896,741107
2,1.058781,4.859403,0.010318,152.779363,0.368714,0.355444,0.313010,1.40625,1.40625,1.40625,5.140045e+06,3.797250e+06,3.138397e+08,0.359043,0.365162,0.336844,661899
3,1.068818,4.836110,0.010475,148.159223,0.369262,0.355497,0.312262,1.40625,1.40625,1.40625,1.933899e+06,3.522100e+06,3.096106e+08,0.368000,0.365229,0.336382,662895
4,1.066041,4.829051,0.009648,143.652839,0.368751,0.354904,0.312821,1.40625,1.40625,1.40625,3.181196e+06,4.933232e+06,3.274322e+08,0.364262,0.364178,0.338324,661898


In [21]:
df_geometric = df.merge(df_sys_geo, how='inner', on='pixel_id')
print(df_geometric.isna().sum())

pixel_id             0
lrg_density          0
elg_density          0
qso_density          0
stellar_density      0
airmass_galaxy       0
fwhm_galaxy          0
ebv_galaxy           0
ccdnphotom_galaxy    0
ccdskysb_galaxy_g    0
ccdskysb_galaxy_r    0
ccdskysb_galaxy_z    0
exptime_galaxy_g     0
exptime_galaxy_r     0
exptime_galaxy_z     0
meansky_galaxy_g     0
meansky_galaxy_r     0
meansky_galaxy_z     0
galdepth_galaxy_g    0
galdepth_galaxy_r    0
galdepth_galaxy_z    0
dtype: int64


#### Geometric Systematics Values, not involving edge pixels

In [22]:
with open('../bricks_data/pixel2systematics_geometric_non_inclusive.pickle', 'rb') as f:
    geometric_dict_no = pickle.load(f)
    f.close()

print(len(geometric_dict_no.keys()))

455672


In [23]:
pixels = df.pixel_id.tolist()
pixel_keys = list(geometric_dict_no.keys())
for pix in pixel_keys:
    if pix in pixels:
        continue
    geometric_dict_no.pop(pix)
print(len(geometric_dict_no.keys()))


3621


In [24]:
sys_list = []
for key in geometric_dict_no.keys():
    ind_sys_list = geometric_dict_no[key]
    ind_sys_list.append(key)
    sys_list.append(ind_sys_list)

df_sys_geo_no = pd.DataFrame(sys_list, columns=['airmass_galaxy',
'fwhm_galaxy',
'ebv_galaxy',
'ccdnphotom_galaxy',

'ccdskysb_galaxy_g',
'ccdskysb_galaxy_r',
'ccdskysb_galaxy_z',
'exptime_galaxy_g',
'exptime_galaxy_r',
'exptime_galaxy_z',
'meansky_galaxy_g',
'meansky_galaxy_r',
'meansky_galaxy_z',
'galdepth_galaxy_g',
'galdepth_galaxy_r',
'galdepth_galaxy_z', 'pixel_id'])



In [25]:
df_geometric_no = df.merge(df_sys_geo_no, how='inner', on='pixel_id')
print(df_geometric_no.isna().sum())

pixel_id             0
lrg_density          0
elg_density          0
qso_density          0
stellar_density      0
airmass_galaxy       0
fwhm_galaxy          0
ebv_galaxy           0
ccdnphotom_galaxy    0
ccdskysb_galaxy_g    0
ccdskysb_galaxy_r    0
ccdskysb_galaxy_z    0
exptime_galaxy_g     0
exptime_galaxy_r     0
exptime_galaxy_z     0
meansky_galaxy_g     0
meansky_galaxy_r     0
meansky_galaxy_z     0
galdepth_galaxy_g    0
galdepth_galaxy_r    0
galdepth_galaxy_z    0
dtype: int64


In [34]:
print(df_geometric.describe())
print(df_geometric_no.describe())

            pixel_id  lrg_density  elg_density  qso_density  stellar_density  \
count    3621.000000  3621.000000  3621.000000  3621.000000      3621.000000   
mean   421882.380558     0.111207     0.262740     0.276077         0.087102   
std    168630.278409     1.024736     0.966438     0.964311         1.347068   
min    166098.000000    -0.899365    -0.969284    -0.972915        -0.897098   
25%    285272.000000    -0.597461    -0.539263    -0.512467        -0.691295   
50%    386229.000000    -0.194922     0.013620     0.029236        -0.279689   
75%    568992.000000     0.610157     0.873662     0.895961         0.337720   
max    757472.000000    15.906644     5.266017     5.771290        14.538136   

       airmass_galaxy  fwhm_galaxy   ebv_galaxy  ccdnphotom_galaxy  \
count     3621.000000  3621.000000  3621.000000        3621.000000   
mean         1.406883     4.556055     0.045843         282.628619   
std          0.254607     0.410849     0.035465         257.617997   

In [26]:
print(df_kitanidis.shape)
print(df_geometric.shape)
print(df_geometric_no.shape)


(1535, 21)
(3621, 21)
(3621, 21)


### Running the Regressions

#### 1. Kitanidis

In [23]:
df_kitanidis.head()
df_kitanidis_reg = df_kitanidis.drop(columns=['pixel_id'])
df_kitanidis_reg.head()

,lrg_density,elg_density,qso_density,stellar_density,airmass_galaxy,fwhm_galaxy,ebv_galaxy,ccdnphotom_galaxy,ccdskysb_galaxy_g,ccdskysb_galaxy_r,ccdskysb_galaxy_z,exptime_galaxy_g,exptime_galaxy_r,exptime_galaxy_z,meansky_galaxy_g,meansky_galaxy_r,meansky_galaxy_z,galdepth_galaxy_g,galdepth_galaxy_r,galdepth_galaxy_z
0,0.509522,1.180820,0.787620,-0.279689,1.615369,4.563708,0.027940,617.485215,23.461968,22.460363,19.328887,143.100496,139.892237,122.275265,0.199914,0.920791,2.869334,23.626332,22.942529,22.215109
1,1.817774,2.133008,1.410579,1.263834,1.551244,4.652137,0.022238,997.009628,23.454327,22.586219,19.084433,158.096504,82.525712,165.945433,0.203127,0.499488,3.490955,23.604491,22.966942,22.331463
2,0.106983,0.750799,0.191747,-0.794197,1.406610,4.458577,0.018477,367.511248,23.441590,22.625744,19.497361,128.317445,87.744524,106.219834,0.213351,0.564238,1.995409,23.598964,22.922293,22.342016
3,0.106983,0.658651,0.083406,0.955130,2.301381,6.055535,0.131243,459.871429,23.432264,22.486569,19.849928,200.000000,124.000000,221.028571,0.126744,0.333307,3.754301,24.007800,23.363166,22.229356
4,1.515870,3.146629,1.329324,-0.485492,1.440663,4.587659,0.018841,373.382383,23.452281,22.653194,19.589021,128.873803,83.743415,107.265548,0.293171,0.540003,6.617020,23.605227,22.900591,22.310904


In [24]:
y_lrg = df_kitanidis_reg['lrg_density']#.to_numpy(copy=True)
y_elg = df_kitanidis_reg['elg_density']#.to_numpy(copy=True)
y_qso = df_kitanidis_reg['qso_density']#.to_numpy(copy=True)

X = df_kitanidis_reg.drop(columns=['lrg_density','elg_density','qso_density'])#.to_numpy(copy=True)

In [25]:
print(len(y_lrg))
print(len(X))

1535
1535


In [26]:


ols_lrg_kitanidis = sm.OLS(y_lrg,X).fit()
print(ols_lrg_kitanidis.summary())



                                 OLS Regression Results                                
Dep. Variable:            lrg_density   R-squared (uncentered):                   0.477
Model:                            OLS   Adj. R-squared (uncentered):              0.471
Method:                 Least Squares   F-statistic:                              81.43
Date:                Wed, 16 Jun 2021   Prob (F-statistic):                   2.67e-199
Time:                        18:08:54   Log-Likelihood:                         -2213.4
No. Observations:                1535   AIC:                                      4461.
Df Residuals:                    1518   BIC:                                      4551.
Df Model:                          17                                                  
Covariance Type:            nonrobust                                                  
                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------

In [27]:
ols_elg_kitanidis = sm.OLS(y_elg,X).fit()
print(ols_elg_kitanidis.summary())

                                 OLS Regression Results                                
Dep. Variable:            elg_density   R-squared (uncentered):                   0.761
Model:                            OLS   Adj. R-squared (uncentered):              0.758
Method:                 Least Squares   F-statistic:                              284.5
Date:                Wed, 16 Jun 2021   Prob (F-statistic):                        0.00
Time:                        18:09:26   Log-Likelihood:                         -1594.0
No. Observations:                1535   AIC:                                      3222.
Df Residuals:                    1518   BIC:                                      3313.
Df Model:                          17                                                  
Covariance Type:            nonrobust                                                  
                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------

In [28]:
ols_qso_kitanidis = sm.OLS(y_qso,X).fit()
print(ols_qso_kitanidis.summary())

                                 OLS Regression Results                                
Dep. Variable:            qso_density   R-squared (uncentered):                   0.781
Model:                            OLS   Adj. R-squared (uncentered):              0.779
Method:                 Least Squares   F-statistic:                              318.6
Date:                Wed, 16 Jun 2021   Prob (F-statistic):                        0.00
Time:                        18:09:37   Log-Likelihood:                         -1534.7
No. Observations:                1535   AIC:                                      3103.
Df Residuals:                    1518   BIC:                                      3194.
Df Model:                          17                                                  
Covariance Type:            nonrobust                                                  
                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------

#### 2. Geometric

In [29]:
df_geometric.head()
df_geometric_reg = df_geometric.drop(columns=['pixel_id'])
df_geometric_reg.head()

,lrg_density,elg_density,qso_density,stellar_density,airmass_galaxy,fwhm_galaxy,ebv_galaxy,ccdnphotom_galaxy,ccdskysb_galaxy_g,ccdskysb_galaxy_r,ccdskysb_galaxy_z,exptime_galaxy_g,exptime_galaxy_r,exptime_galaxy_z,meansky_galaxy_g,meansky_galaxy_r,meansky_galaxy_z,galdepth_galaxy_g,galdepth_galaxy_r,galdepth_galaxy_z
0,0.308252,0.075052,0.056321,-0.691295,1.220610,3.898208,0.016500,1550.658093,0.364892,0.351523,0.301166,1.515938,1.309821,1.509230,0.003598,0.011705,0.032974,0.366218,0.355911,0.348341
1,0.408887,-0.416400,0.110491,0.131917,1.219311,3.862342,0.016510,1587.507883,0.364887,0.351523,0.301310,1.515938,1.309821,1.427000,0.003594,0.011709,0.036043,0.366503,0.356100,0.346743
2,-0.496826,0.412925,-0.024934,-0.485492,1.655938,4.672048,0.028977,579.146330,0.366651,0.351116,0.302868,2.300308,2.244602,1.939581,0.003221,0.015671,0.079584,0.368576,0.358520,0.344488
3,-0.496826,-0.293537,-0.133275,-0.794197,1.658746,4.609840,0.028156,577.106054,0.366584,0.351225,0.304636,2.370826,2.023661,1.915658,0.002980,0.016154,0.094865,0.369514,0.357950,0.342820
4,0.509522,1.180820,0.787620,-0.279689,1.657476,4.666779,0.028076,585.977177,0.366563,0.351269,0.303110,2.371346,2.025000,1.950375,0.002980,0.012420,0.047528,0.369459,0.358867,0.346243


In [30]:
y_lrg = df_geometric_reg['lrg_density']#.to_numpy(copy=True)
y_elg = df_geometric_reg['elg_density']#.to_numpy(copy=True)
y_qso = df_geometric_reg['qso_density']#.to_numpy(copy=True)

X = df_geometric_reg.drop(columns=['lrg_density','elg_density','qso_density'])#.to_numpy(copy=True)

In [31]:
print(len(y_lrg))
print(len(X))

3621
3621


In [31]:
ols_lrg_geometric = sm.OLS(y_lrg,X).fit()
print(ols_lrg_geometric.summary())
print()
ols_elg_geometric = sm.OLS(y_elg,X).fit()
print(ols_elg_geometric.summary())
print()
ols_qso_geometric = sm.OLS(y_qso,X).fit()
print(ols_qso_geometric.summary())


                                 OLS Regression Results                                
Dep. Variable:            lrg_density   R-squared (uncentered):                   0.312
Model:                            OLS   Adj. R-squared (uncentered):              0.309
Method:                 Least Squares   F-statistic:                              96.06
Date:                Fri, 18 Jun 2021   Prob (F-statistic):                   1.04e-276
Time:                        13:34:38   Log-Likelihood:                         -4570.5
No. Observations:                3621   AIC:                                      9175.
Df Residuals:                    3604   BIC:                                      9280.
Df Model:                          17                                                  
Covariance Type:            nonrobust                                                  
                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------

### 3. Geometric, non-inclusive of edge CCDs

In [32]:
df_geometric_no_reg = df_geometric_no.drop(columns=['pixel_id'])


y_lrg = df_geometric_no_reg['lrg_density']#.to_numpy(copy=True)
y_elg = df_geometric_no_reg['elg_density']#.to_numpy(copy=True)
y_qso = df_geometric_no_reg['qso_density']#.to_numpy(copy=True)

X = df_geometric_no_reg.drop(columns=['lrg_density','elg_density','qso_density'])#.to_numpy(copy=True)

In [33]:
ols_lrg_geometric_no = sm.OLS(y_lrg,X).fit()
print(ols_lrg_geometric_no.summary())
ols_elg_geometric_no = sm.OLS(y_elg,X).fit()
print(ols_elg_geometric_no.summary())
ols_qso_geometric_no = sm.OLS(y_qso,X).fit()
print(ols_qso_geometric_no.summary())


                                 OLS Regression Results                                
Dep. Variable:            lrg_density   R-squared (uncentered):                   0.312
Model:                            OLS   Adj. R-squared (uncentered):              0.309
Method:                 Least Squares   F-statistic:                              96.06
Date:                Fri, 18 Jun 2021   Prob (F-statistic):                   1.04e-276
Time:                        13:37:30   Log-Likelihood:                         -4570.5
No. Observations:                3621   AIC:                                      9175.
Df Residuals:                    3604   BIC:                                      9280.
Df Model:                          17                                                  
Covariance Type:            nonrobust                                                  
                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------